In [17]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATASETS
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE CELL.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote
from urllib.error import HTTPError
from zipfile import ZipFile

CHUNK_SIZE = 40960 
DATASET_MAPPING = 'ckplus:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F65125%2F128470%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20210321%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20210321T233051Z%26X-Goog-Expires%3D259199%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3fe73639e5c3f7848c417a8d49ab363474682e26fc9f04c212bb0cc8ca070fa20799b462a139a7e8a5626148ce5e2dbc01980a3cadec1972c302e3f06eeda2461726bdd7301c12154e622442923a1358c95d644e31cc1c3a92a047e170c5d0ef0806842e37d951601059f8a3e663f045440b8b3abaaad5e0634a0471110b84cb2c59416d8a6101e3d2b5cb56f7ff0972d96c213fc73e46fe3d8aa2f62bc502c15cd0f37f55418799aaf0ff0a5e884b83685670d0b16745d22d6de0f78f1cfc1e6edb1c4486695a5bfa9caa913e28f4bbfb5424c068d0bcc4c7963e45e8c7354b25fce9c20c242d2e46f796d96a84e42c6d357cbccb4709fdc185f434c5937dd4'
KAGGLE_INPUT_PATH='/home/kaggle/input'
KAGGLE_INPUT_SYMLINK='/kaggle'

os.makedirs(KAGGLE_INPUT_PATH, 777)
os.symlink(KAGGLE_INPUT_PATH, os.path.join('..', 'input'), target_is_directory=True)
os.makedirs(KAGGLE_INPUT_SYMLINK)
os.symlink(KAGGLE_INPUT_PATH, os.path.join(KAGGLE_INPUT_SYMLINK, 'input'), target_is_directory=True)

for dataset_mapping in DATASET_MAPPING.split(','):
    directory, download_url_encoded = dataset_mapping.split(':')
    download_url = unquote(download_url_encoded)
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as zipfileres, NamedTemporaryFile() as tfile:
            total_length = zipfileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes zipped')
            dl = 0
            data = zipfileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = zipfileres.read(CHUNK_SIZE)
            print(f'\nUnzipping {directory}')
            with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue
print('Dataset import complete.')


FileExistsError: [Errno 17] File exists: '/home/kaggle/input'

In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams

## Import and Pre-process Data

In [19]:
data_path = '../input/ckplus/CK+48'
data_dir_list = os.listdir(data_path)

img_rows=256
img_cols=256
num_channel=1

num_epoch=10

img_data_list=[]


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        #input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(48,48))
        img_data_list.append(input_img_resize)
        
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data = img_data/255
img_data.shape

Loaded the images of dataset-disgust

Loaded the images of dataset-sadness

Loaded the images of dataset-happy

Loaded the images of dataset-anger

Loaded the images of dataset-fear

Loaded the images of dataset-surprise

Loaded the images of dataset-contempt



(981, 48, 48, 3)

In [20]:
num_classes = 7

num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:134]=0 #135
labels[135:188]=1 #54
labels[189:365]=2 #177
labels[366:440]=3 #75
labels[441:647]=4 #207
labels[648:731]=5 #84
labels[732:980]=6 #249

names = ['anger','contempt','disgust','fear','happy','sadness','surprise']

def getLabel(id):
    return ['anger','contempt','disgust','fear','happy','sadness','surprise'][id]

In [22]:
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
x_test=X_test

In [5]:
## Transfer Model Definition

In [6]:
from keras.applications import VGG16
#Load the VGG model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
base_model.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
______________________________________________________________

In [9]:
import numpy as np
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.utils import plot_model
def define_model_original(base_model):
    inputs1 = Input(shape=(None, None, 3,))
    # Create the model
    model = Sequential()
 
    # Add the vgg convolutional base model
    model.add(base_model)
 
    # Add new layers
    model.add(Flatten())
    model.add(Dense(units=1024,activation="relu"))
    model.add(Dense(units=1024,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation="softmax"))
    
    # summarize model
    print(model.summary())
    plot_model(model, to_file='mode_original.png', show_shapes=True)
    return model 

def define_model_modified(base_model):
    inputs1 = Input(shape=(None, None, 3,))
    # Create the model
    model = Sequential()
 
    # Add the vgg convolutional base model
    model.add(base_model)
 
    # Add new layers
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(units=1000,activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation="softmax"))
    
    # summarize model
    print(model.summary())
    plot_model(model, to_file='mode_modified.png', show_shapes=True)
    return model 

model_original = define_model_original(base_model)
model_modified = define_model_modified(base_model)
model_original.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
model_modified.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_8 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 7)                 7175      
Total params: 16,296,775
Trainable params: 16,296,775
Non-trainable params: 0
__________________________________________

In [23]:
## Conduct k-Fold Cross-Validation

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=False)

from keras.preprocessing.image import ImageDataGenerator
# Any results you write to the current directory are saved as output

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.datasets import make_classification
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")
BS = 8
EPOCHS = 200
result = []
scores_loss = []
scores_acc = []
k_no = 0

def fit_model(model, file_path, X_Train_, Y_Train, X_Test_, Y_Test):
    checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')
    early = EarlyStopping(monitor="loss", mode="min", patience=8)
    callbacks_list = [checkpoint, early]
    hist = model.fit_generator(aug.flow(X_Train_, Y_Train), epochs=EPOCHS,validation_data=(X_Test_, Y_Test), callbacks=callbacks_list, verbose=1)
    model.load_weights(file_path)
    result.append(model.predict(X_Test_))
    score = model.evaluate(X_Test_,Y_Test, verbose=0)
    scores_loss.append(score[0])
    scores_acc.append(score[1])
    
for train_index, test_index in kf.split(x):
    X_Train_ = x[train_index]
    Y_Train = y[train_index]
    X_Test_ = x[test_index]
    Y_Test = y[test_index]

    file_path_original = "/kaggle/working/weights_original_"+str(k_no)+".hdf5"
    file_path_modified = "/kaggle/working/weights_modified_"+str(k_no)+".hdf5"
    fit_model(model_original, file_path_original, X_Train_, Y_Train, X_Test_, Y_Test)
    fit_model(model_modified, file_path_modified, X_Train_, Y_Train, X_Test_, Y_Test)
    k_no+=1

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
25/25 [==============================] - 51s 2s/step - loss: 3.3391 - acc: 0.2432 - val_loss: 1.8508 - val_acc: 0.2081

Epoch 00001: loss improved from inf to 2.64287, saving model to /kaggle/working/weights_original_0.hdf5
Epoch 2/200
25/25 [==============================] - 49s 2s/step - loss: 1.8521 - acc: 0.2692 - val_loss: 1.8812 - val_acc: 0.2081

Epoch 00002: loss improved from 2.64287 to 1.88650, saving model to /kaggle/working/weights_original_0.hdf5
Epoch 3/200
25/25 [==============================] - 48s 2s/step - loss: 1.8635 - acc: 0.2124 - val_loss: 1.8769 - val_acc: 0.2081

Epoch 00003: loss improved from 1.88650 to 1.84964, saving model to /kaggle/working/weights_original_0.hdf5
Epoch 4/200
25/25 [==============================] - 48s 2s/step - loss: 1.8069 - acc: 0.2877 - val_loss: 1.8708 - val_acc: 0.2081

Epoch 00004: loss improved from 1.84964 to 1.83320, saving model to /kaggle/working/weights_original_0.hdf5
Epoch 5/200
25/25 [=========================